# CeneoScarper

## Lista kroków "manualnych"

1. Otworzyć Ceneo
2. Wyszukać produkt
3. Otworzyć okno produktu
4. Wybrać zakładkę Opinie i Recencje
5. Skopiować opinie
6. Wkleić opinie

## Lista kroków scrapera

1. Wysłanie żądania dostępu do zasobu
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Wydobycie z kodu opinii jej składowych do słownika dla każdej opinii na stronie
4. Dodanie słownika reprezentującego pojedynczą opinię do listy
5. Przejdź do kolejnej strony z opiniami
6. Powtarzaj kroki 1-5 dopóki są strony z opiniami
7. Zapis listy słowników do pliku JSON

## Struktura opinii w serwisie Ceneo.pl

|składowa|nazwa|selektor|
|--------|-----|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autor|author|span.user-post__author-name|
|rekomendacja|recommendation|span.user-post__author-recomendation > em|
|liczba gwiazdek|stars|span.user-post__score-count|
|treść opinii|content|div.user-post__text|
|lista zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
|lista wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|data wystawienia opinii|post_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|useful|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|useless|button.vote-no > span|

49729622
61161229
36068712
37089065
92504536

1. Import bibliotek

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup

2. Definicje funkcji
- funkcja do pobierania danych ze strony WWW

In [ ]:
def get_data(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if attribute:
        if selector:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        return ancestor[attribute].strip()
    try:
        return ancestor.select_one(selector).text.strip()
    except AttributeError:
        return None

3. Definicje stałych i zmiennych globalnych
- zmienna opisująca strukturę opinii w serwisie Ceneo.pl

In [ ]:
selectors = {
    'opinion_id': (None, "data-entry-id"),
    'author': ("span.user-post__author-name",),
    'recommendation': ("span.user-post__author-recomendation > em",),
    'stars': ("span.user-post__score-count",),
    'content': ("div.user-post__text",),
    'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    'post_date': ("span.user-post__published > time:nth-child(1)", "datetime"),
    'purchase_date': ("span.user-post__published > time:nth-child(2)", "datetime"),
    'useful': ("button.vote-yes > span",),
    'useless': ("button.vote-no > span",),
}

4. Podanie adresu pierwszej strony z opiniami na podstawie kodu produktu

In [ ]:
product_code = input("Podaj kod produktu")
url = f"https://www.ceneo.pl/{product_code}#tab=reviews"

5. Pobranie wszystkich opinii o produkcie

In [ ]:
all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: get_data(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://ceneo.pl"+page.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

6. Zapis opinii do pliku JSON

In [ ]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
file = open(f"opinions/{product_code}.json", "w", encoding="UTF-8")
json.dump(all_opinions, file, indent=4, ensure_ascii=False)
file.close()